<img src="https://deecamp.chuangxin.com/assets/image/logo_nav_zh.jpg" width="40%">

In [2]:
import pandas as pd
import csv

In [16]:
! ls /data/jupyter_root/ZJ/test1.0 

block_1.attributes  block_2.attributes	block_3.attributes
block_1.tuples	    block_2.tuples	block_3.tuples


## 定义一个函数，先输出原始feature文件的P & R，再输出通过D-Cube分块的P & R

名称 | 定义
 --- | ---
TP	| 真实类别为positive，模型预测的类别也为positive
FP	| 预测为positive，但真实类别为negative，真实类别和预测类别不一致
FN	| 预测为negative，但真实类别为positive，真实类别和预测类别不一致
TN	| 真实类别为negative，模型预测的类别也为negative

准确率（accuracy）计算公式如下所示 不计算这个

\begin{equation}\label{equ:accuracy} \mbox{accuracy} = \frac{TP+TN}{TP+TN+FP+FN} = \frac{TP+TN }{\mbox{all data}} \end{equation}

positive class的精确率（precision）计算公式如下

\begin{equation}\label{equ:precision} \mbox{precision} = \frac{TP}{TP+FP} = \frac{TP}{\mbox{预测为positive的样本}} \end{equation}

positive class的召回率（recall）计算公式如下

\begin{equation}\label{equ:recall} \mbox{recall} = \frac{TP}{TP+FN} = \frac{TP}{\mbox{真实为positive的样本}} \end{equation}

In [8]:
def get_PR(f_count,j,choice):#f_count：the index of feature;j: index of test; choice: index of block
    
    #准备用于读取数据的 pandas头（防止第一行数据成为标签）
    data_path="/data/jupyter_root/dcube_data/feature"#feature文件位置
    #try:
    cols = pd.read_csv(data_path+str(f_count)+"/test"+str(j)+"/blocks.txt",nrows=1).columns#先读第一行，用于提取数据维度
    #except e:
    #    print(e)
    #    print("false1")
    #    return 0        

    user_index=['user_name']#初始化一个等待补全的names
    #print(cols.shape[0]-1)
    for i in range(1,cols.shape[0]-1):
            user_index.append(str(i))
    user_index.append('count')#补上最后的count
    
    #开始读取数据
    #result=pd.read_csv(data_path+str(f_count)+"/test"+str(j)+"/block_1.tuples",sep=',',names=user_index)#读取全部的数据，并给上names
    #try:
    if(choice=='all'):
        result=pd.read_csv(data_path+str(f_count)+"/test"+str(j)+"/blocks.txt",sep=',',names=user_index)
    elif(choice==1 or choice==2 or choice==3 ):
        result=pd.read_csv(data_path+str(f_count)+"/test"+str(j)+"/block_"+str(choice)+".tuples",sep=',',names=user_index)
    elif(choice==12 or choice==13 or choice==23 or choice ==123 ):
        result=pd.read_csv(data_path+str(f_count)+"/test"+str(j)+"/block"+str(choice)+".txt",sep=',',names=user_index)
    #except:
    #    print("false2")
    #    return 0
    names = result['user_name'].drop_duplicates()#预测出坏用户的用户列表
    #print(names)
    dataset = pd.read_csv(open('/data/csv/label_eventsV1.csv','r',encoding = 'gb18030'))#读取总标签数据
    data = dataset[dataset['user_name'].isin(names)][['user_name','label']]#预测出坏用户的真实标签分布
    data = data.drop_duplicates()#去重
    #print(data)

    
    raw_result=pd.read_csv(data_path+str(f_count)+".txt",sep=',',names=user_index)#读取全部的原始数据，并给上names
    raw_names = raw_result['user_name'].drop_duplicates()#原始数据中用户的用户列表
    #print(raw_names)
    raw_data = dataset[dataset['user_name'].isin(raw_names)][['user_name','label']]#原始数据中坏用户的真实标签分布
    raw_data = raw_data.drop_duplicates()#去重
    #print(raw_data)
    
    
    TP=data.label.sum()
    if(TP==0):
        return 0
    P=raw_data.label.sum()
    FP=data.label.count()-data.label.sum()
    N=raw_data.label.count()-raw_data.label.sum()
    Precision = (TP/P)/(TP/P+FP/N) #精确率计算

    #data_label_1 = dataset[dataset['label']==1][['user_name','label']] #原数据中label为1的坏用户
    #data_label_1 = data_label_1.drop_duplicates()#去重
    Recall = data.label.sum()/raw_data.label.sum() # 召回率计算
    print("TP",TP,"P",P,"FP",FP,"N",N)
    print('测试特征'+str(f_count)+"_"+str(j)+str(choice))
    print('\t\t精确率 Precision\t= '+str(Precision))
    print('\t\t召回率 Recall   \t= '+str(Recall))
#get_PR(1,1)

In [12]:
for i in range(1,135):
    #for j in range(3,11):
        j=3
        try:
            #if(j==3):
                get_PR(i,j,1)
                get_PR(i,j,2)
                get_PR(i,j,3)
                get_PR(i,j,'all')
                get_PR(i,j,12)
                get_PR(i,j,13)
                get_PR(i,j,23)
                get_PR(i,j,123)
        except:
            print("false")
            continue

TP 1 P 5538 FP 0 N 3871
测试特征1_31
		精确率 Precision	= 1.0
		召回率 Recall   	= 0.00018057060310581438
TP 1 P 5538 FP 0 N 3871
测试特征1_3all
		精确率 Precision	= 1.0
		召回率 Recall   	= 0.00018057060310581438
TP 1 P 5538 FP 1 N 3871
测试特征1_312
		精确率 Precision	= 0.4114146030396429
		召回率 Recall   	= 0.00018057060310581438
TP 1 P 5538 FP 1 N 3871
测试特征1_313
		精确率 Precision	= 0.4114146030396429
		召回率 Recall   	= 0.00018057060310581438
TP 1 P 5538 FP 2 N 3871
测试特征1_3123
		精确率 Precision	= 0.2589817354653108
		召回率 Recall   	= 0.00018057060310581438
TP 2078 P 5538 FP 1516 N 3871
测试特征2_31
		精确率 Precision	= 0.4893041450171434
		召回率 Recall   	= 0.37522571325388226
TP 284 P 5538 FP 298 N 3871
测试特征2_32
		精确率 Precision	= 0.39981408799834744
		召回率 Recall   	= 0.05128205128205128
TP 2774 P 5538 FP 1818 N 3871
测试特征2_33
		精确率 Precision	= 0.5161026226846006
		召回率 Recall   	= 0.5009028530155291
TP 2078 P 5538 FP 1516 N 3871
测试特征2_3all
		精确率 Precision	= 0.4893041450171434
		召回率 Recall   	= 0.37522571325388226
TP 2119 P 553

TP 2987 P 5538 FP 2021 N 3871
测试特征10_312
		精确率 Precision	= 0.5081384180821996
		召回率 Recall   	= 0.5393643914770675
TP 1216 P 5538 FP 1022 N 3871
测试特征10_313
		精确率 Precision	= 0.4540511925758071
		召回率 Recall   	= 0.2195738533766703
TP 2010 P 5538 FP 1331 N 3871
测试特征10_323
		精确率 Precision	= 0.5135176125748327
		召回率 Recall   	= 0.3629469122426869
TP 3008 P 5538 FP 2039 N 3871
测试特征10_3123
		精确率 Precision	= 0.5076732378645751
		召回率 Recall   	= 0.5431563741422897
TP 257 P 5538 FP 249 N 3871
测试特征12_31
		精确率 Precision	= 0.41909311153508977
		召回率 Recall   	= 0.046406644998194295
TP 1553 P 5538 FP 887 N 3871
测试特征12_32
		精确率 Precision	= 0.5503236078718996
		召回率 Recall   	= 0.2804261466233297
TP 2165 P 5538 FP 1429 N 3871
测试特征12_33
		精确率 Precision	= 0.5143273040863992
		召回率 Recall   	= 0.3909353557240881
TP 257 P 5538 FP 249 N 3871
测试特征12_3all
		精确率 Precision	= 0.41909311153508977
		召回率 Recall   	= 0.046406644998194295
TP 1773 P 5538 FP 1076 N 3871
测试特征12_312
		精确率 Precision	= 0.5352668436569751
		

TP 3401 P 5538 FP 2009 N 3871
测试特征35_313
		精确率 Precision	= 0.5419789122054638
		召回率 Recall   	= 0.6141206211628747
TP 2344 P 5538 FP 1394 N 3871
测试特征35_323
		精确率 Precision	= 0.5403026248815322
		召回率 Recall   	= 0.4232574936800289
TP 3401 P 5538 FP 2009 N 3871
测试特征35_3123
		精确率 Precision	= 0.5419789122054638
		召回率 Recall   	= 0.6141206211628747
TP 2 P 5538 FP 2 N 3871
测试特征37_31
		精确率 Precision	= 0.4114146030396429
		召回率 Recall   	= 0.00036114120621162876
TP 248 P 5538 FP 201 N 3871
测试特征37_32
		精确率 Precision	= 0.4630682064842515
		召回率 Recall   	= 0.04478150957024196
TP 2 P 5538 FP 2 N 3871
测试特征37_3all
		精确率 Precision	= 0.4114146030396429
		召回率 Recall   	= 0.00036114120621162876
TP 248 P 5538 FP 201 N 3871
测试特征37_312
		精确率 Precision	= 0.4630682064842515
		召回率 Recall   	= 0.04478150957024196
TP 2 P 5538 FP 2 N 3871
测试特征37_313
		精确率 Precision	= 0.4114146030396429
		召回率 Recall   	= 0.00036114120621162876
TP 248 P 5538 FP 201 N 3871
测试特征37_323
		精确率 Precision	= 0.4630682064842515
		召回率 Recall

TP 220 P 5538 FP 181 N 3871
测试特征47_32
		精确率 Precision	= 0.45934245883760394
		召回率 Recall   	= 0.03972553268327916
TP 1741 P 5538 FP 820 N 3871
测试特征47_33
		精确率 Precision	= 0.5974352716719747
		召回率 Recall   	= 0.31437342000722285
TP 975 P 5538 FP 550 N 3871
测试特征47_3all
		精确率 Precision	= 0.5533952823445317
		召回率 Recall   	= 0.176056338028169
TP 1124 P 5538 FP 641 N 3871
测试特征47_312
		精确率 Precision	= 0.5506999109717395
		召回率 Recall   	= 0.20296135789093536
TP 2636 P 5538 FP 1310 N 3871
测试特征47_313
		精确率 Precision	= 0.5844613264098844
		召回率 Recall   	= 0.4759841097869267
TP 1906 P 5538 FP 966 N 3871
测试特征47_323
		精确率 Precision	= 0.5796843359207859
		召回率 Recall   	= 0.3441675695196822
TP 2730 P 5538 FP 1366 N 3871
测试特征47_3123
		精确率 Precision	= 0.5828038766125667
		召回率 Recall   	= 0.49295774647887325
TP 4 P 5538 FP 3 N 3871
测试特征48_31
		精确率 Precision	= 0.48239765717490185
		召回率 Recall   	= 0.0007222824124232575
TP 1064 P 5538 FP 563 N 3871
测试特征48_32
		精确率 Precision	= 0.5691515866898413
		召回率 Reca

TP 3244 P 5538 FP 1666 N 3871
测试特征57_3all
		精确率 Precision	= 0.5764607439131921
		召回率 Recall   	= 0.5857710364752619
TP 5503 P 5538 FP 3692 N 3871
测试特征57_312
		精确率 Precision	= 0.5102496959824265
		召回率 Recall   	= 0.9936800288912965
TP 3247 P 5538 FP 1703 N 3871
测试特征57_313
		精确率 Precision	= 0.5713152940150819
		召回率 Recall   	= 0.5863127482845792
TP 2438 P 5538 FP 2278 N 3871
测试特征57_323
		精确率 Precision	= 0.42794501734044915
		召回率 Recall   	= 0.4402311303719754
TP 5503 P 5538 FP 3703 N 3871
测试特征57_3123
		精确率 Precision	= 0.5095062400789183
		召回率 Recall   	= 0.9936800288912965
TP 1847 P 5538 FP 981 N 3871
测试特征58_31
		精确率 Precision	= 0.568227973501323
		召回率 Recall   	= 0.33351390393643915
TP 3538 P 5538 FP 2114 N 3871
测试特征58_32
		精确率 Precision	= 0.5391345801448887
		召回率 Recall   	= 0.6388587937883713
TP 485 P 5538 FP 1019 N 3871
测试特征58_33
		精确率 Precision	= 0.24963709952468247
		召回率 Recall   	= 0.08757674250631997
TP 1847 P 5538 FP 981 N 3871
测试特征58_3all
		精确率 Precision	= 0.568227973501323
		召

...

TP 1467 P 4800 FP 778 N 2608
测试特征89_313
		精确率 Precision	= 0.5060536992006703
		召回率 Recall   	= 0.305625
TP 4030 P 4800 FP 2244 N 2608
测试特征89_323
		精确率 Precision	= 0.4938688359434324
		召回率 Recall   	= 0.8395833333333333
TP 4600 P 4800 FP 2485 N 2608
测试特征89_3123
		精确率 Precision	= 0.5014378385608239
		召回率 Recall   	= 0.9583333333333334
TP 1483 P 4800 FP 782 N 2608
测试特征90_31
		精确率 Precision	= 0.5074832731158506
		召回率 Recall   	= 0.30895833333333333
TP 3675 P 4800 FP 1985 N 2608
测试特征90_32
		精确率 Precision	= 0.5014754818861054
		召回率 Recall   	= 0.765625
TP 572 P 4800 FP 337 N 2608
测试特征90_33
		精确率 Precision	= 0.4797670014819694
		召回率 Recall   	= 0.11916666666666667
TP 1483 P 4800 FP 782 N 2608
测试特征90_3all
		精确率 Precision	= 0.5074832731158506
		召回率 Recall   	= 0.30895833333333333
TP 4178 P 4800 FP 2216 N 2608
测试特征90_312
		精确率 Precision	= 0.5060238636245424
		召回率 Recall   	= 0.8704166666666666
TP 1774 P 4800 FP 918 N 2608
测试特征90_313
		精确率 Precision	= 0.5121882096209096
		召回率 Recall   	= 0.369583

TP 1616 P 5538 FP 1952 N 3871
测试特征98_323
		精确率 Precision	= 0.3665558167159975
		召回率 Recall   	= 0.29180209461899603
TP 5506 P 5538 FP 3762 N 3871
测试特征98_3123
		精确率 Precision	= 0.5056915348274669
		召回率 Recall   	= 0.994221740700614
TP 5466 P 5538 FP 3660 N 3871
测试特征99_31
		精确率 Precision	= 0.5107391948714064
		召回率 Recall   	= 0.9869989165763814
TP 283 P 5538 FP 175 N 3871
测试特征99_32
		精确率 Precision	= 0.5305968150426006
		召回率 Recall   	= 0.05110148067894547
TP 233 P 5538 FP 846 N 3871
测试特征99_33
		精确率 Precision	= 0.16143338277468544
		召回率 Recall   	= 0.04207295052365475
TP 5466 P 5538 FP 3660 N 3871
测试特征99_3all
		精确率 Precision	= 0.5107391948714064
		召回率 Recall   	= 0.9869989165763814
TP 5516 P 5538 FP 3693 N 3871
测试特征99_312
		精确率 Precision	= 0.5107716508385387
		召回率 Recall   	= 0.9960274467316721
TP 5485 P 5538 FP 3809 N 3871
测试特征99_313
		精确率 Precision	= 0.5016324574688441
		召回率 Recall   	= 0.9904297580353918
TP 509 P 5538 FP 995 N 3871
测试特征99_323
		精确率 Precision	= 0.2633914517309928
		召回率 

TP 511 P 4322 FP 342 N 2784
测试特征107_323
		精确率 Precision	= 0.49043350197948943
		召回率 Recall   	= 0.11823229986117538
TP 4309 P 4322 FP 2769 N 2784
测试特征107_3123
		精确率 Precision	= 0.5005975244236387
		召回率 Recall   	= 0.9969921332716335
TP 3896 P 4322 FP 2487 N 2784
测试特征108_31
		精确率 Precision	= 0.5022609109269166
		召回率 Recall   	= 0.901434521055067
TP 1790 P 4322 FP 1109 N 2784
测试特征108_32
		精确率 Precision	= 0.5097306202307625
		召回率 Recall   	= 0.4141601110596946
TP 4322 P 4322 FP 2784 N 2784
测试特征108_33
		精确率 Precision	= 0.5
		召回率 Recall   	= 1.0
TP 3896 P 4322 FP 2487 N 2784
测试特征108_3all
		精确率 Precision	= 0.5022609109269166
		召回率 Recall   	= 0.901434521055067
TP 4322 P 4322 FP 2784 N 2784
测试特征108_312
		精确率 Precision	= 0.5
		召回率 Recall   	= 1.0
TP 4322 P 4322 FP 2784 N 2784
测试特征108_313
		精确率 Precision	= 0.5
		召回率 Recall   	= 1.0
TP 4322 P 4322 FP 2784 N 2784
测试特征108_323
		精确率 Precision	= 0.5
		召回率 Recall   	= 1.0
TP 4322 P 4322 FP 2784 N 2784
测试特征108_3123
		精确率 Precision	= 0.5
		召回率 Recall  

TP 3761 P 5538 FP 2155 N 3871
测试特征121_3all
		精确率 Precision	= 0.5495304251604589
		召回率 Recall   	= 0.6791260382809678
TP 5403 P 5538 FP 3674 N 3871
测试特征121_312
		精确率 Precision	= 0.5068877679926566
		召回率 Recall   	= 0.9756229685807151
TP 4492 P 5538 FP 2744 N 3871
测试特征121_313
		精确率 Precision	= 0.5336393000860156
		召回率 Recall   	= 0.8111231491513181
TP 4567 P 5538 FP 3211 N 3871
测试特征121_323
		精确率 Precision	= 0.4985383956487869
		召回率 Recall   	= 0.8246659443842542
TP 5517 P 5538 FP 3838 N 3871
测试特征121_3123
		精确率 Precision	= 0.5011905686377728
		召回率 Recall   	= 0.9962080173347779
TP 3795 P 5538 FP 2180 N 3871
测试特征122_31
		精确率 Precision	= 0.5489029093401651
		召回率 Recall   	= 0.6852654387865655
TP 4492 P 5538 FP 3136 N 3871
测试特征122_32
		精确率 Precision	= 0.5003073452699844
		召回率 Recall   	= 0.8111231491513181
TP 1760 P 5538 FP 1069 N 3871
测试特征122_33
		精确率 Precision	= 0.5350597757871975
		召回率 Recall   	= 0.3178042614662333
TP 3795 P 5538 FP 2180 N 3871
测试特征122_3all
		精确率 Precision	= 0.5489029093

TP 3400 P 4800 FP 1741 N 2608
测试特征130_313
		精确率 Precision	= 0.5148165350673479
		召回率 Recall   	= 0.7083333333333334
TP 3323 P 4800 FP 1907 N 2608
测试特征130_323
		精确率 Precision	= 0.4863295051218901
		召回率 Recall   	= 0.6922916666666666
TP 4770 P 4800 FP 2589 N 2608
测试特征130_3123
		精确率 Precision	= 0.5002605825467601
		召回率 Recall   	= 0.99375
TP 2933 P 4800 FP 1476 N 2608
测试特征131_31
		精确率 Precision	= 0.5191550681468466
		召回率 Recall   	= 0.6110416666666667
TP 3056 P 4800 FP 1758 N 2608
测试特征131_32
		精确率 Precision	= 0.4857283272616642
		召回率 Recall   	= 0.6366666666666667
TP 1106 P 4800 FP 658 N 2608
测试特征131_33
		精确率 Precision	= 0.4773325425362346
		召回率 Recall   	= 0.23041666666666666
TP 2933 P 4800 FP 1476 N 2608
测试特征131_3all
		精确率 Precision	= 0.5191550681468466
		召回率 Recall   	= 0.6110416666666667
TP 4531 P 4800 FP 2450 N 2608
测试特征131_312
		精确率 Precision	= 0.5012055894833779
		召回率 Recall   	= 0.9439583333333333
TP 3364 P 4800 FP 1711 N 2608
测试特征131_313
		精确率 Precision	= 0.5164991258741258
		召回率

# 精确率Precision的计算

# 召回率Recall的计算

In [13]:
pd.set_option('max_row',200) 
pd.set_option('display.max_colwidth', 100)
pd.read_csv("/data/csv/feature_list.csv",sep=',')

,Unnamed: 0,Feature_combination,Standard,Poly,user_name_dic,user_name_num
0,feature1,['user_name'],user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
1,feature2,"['user_name', 'time_stamp_day']",user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
2,feature3,"['user_name', 'time_stamp_hour']",user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
3,feature4,"['user_name', 'time_stamp_3hour']",user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
4,feature5,"['user_name', 'time_stamp_6hour']",user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
5,feature6,['user_name'],os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
6,feature7,"['user_name', 'time_stamp_day']",os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
7,feature8,"['user_name', 'time_stamp_hour']",os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
8,feature9,"['user_name', 'time_stamp_3hour']",os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
9,feature10,"['user_name', 'time_stamp_6hour']",os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
